In [1]:
from langdetect import detect
import time
import pandas as pd
import spacy
import gensim
import os
import numpy as np
import hdbscan
import plotly.graph_objs as go
import nltk
nltk.download('omw')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from collections import Counter
from tqdm import tqdm
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from kneed import KneeLocator
from unidecode import unidecode
from collections import Counter
import re

[nltk_data] Downloading package omw to C:\Users\Giovanni
[nltk_data]     Sacheli\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Giovanni
[nltk_data]     Sacheli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Giovanni Sacheli\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Giovanni
[nltk_data]     Sacheli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Giovanni
[nltk_data]     Sacheli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# stampa numero di core della CPU, valore da usare in workers
import multiprocessing
num_cores = multiprocessing.cpu_count()
print(f'Il numero di core CPU disponibili è: {num_cores}')

Il numero di core CPU disponibili è: 16


In [3]:
# scelta lingua

valid_languages = ['it', 'en', 'es', 'de', 'fr']
linguaz = ""

while linguaz not in valid_languages:
    linguaz = input("Lingua delle kw: it, en, es, de, fr? ")
    if linguaz not in valid_languages:
        print("Lingua non valida. Inserire 'it', 'en', 'es', 'de' o 'fr'.")

if linguaz == "en":
    stemmer = PorterStemmer()
    stopwords = stopwords.words('english')
    nlp = spacy.load("en_core_web_lg")
elif linguaz == "it":
    stemmer = SnowballStemmer("italian")
    stopwords = stopwords.words('italian')
    nlp = spacy.load("it_core_news_lg")
elif linguaz == "es":
    stemmer = SnowballStemmer("spanish")
    stopwords = stopwords.words('spanish')
    nlp = spacy.load("es_core_news_lg")
elif linguaz == "de":
    stemmer = SnowballStemmer("german")
    stopwords = stopwords.words('german')
    nlp = spacy.load("de_core_news_lg")
elif linguaz == "fr":
    stemmer = SnowballStemmer("french")
    stopwords = stopwords.words('french')
    nlp = spacy.load("fr_core_news_lg")

print(f"Lingua scelta: {linguaz}")
print(f"Stemmer: {stemmer}")
print(f"Stopwords: {stopwords}")
print(f"NLP: {nlp}")


Lingua scelta: it
Stemmer: <nltk.stem.snowball.SnowballStemmer object at 0x0000018F2EB5F520>
Stopwords: ['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl', 'alla', 'alle', 'con', 'col', 'coi', 'da', 'dal', 'dallo', 'dai', 'dagli', 'dall', 'dagl', 'dalla', 'dalle', 'di', 'del', 'dello', 'dei', 'degli', 'dell', 'degl', 'della', 'delle', 'in', 'nel', 'nello', 'nei', 'negli', 'nell', 'negl', 'nella', 'nelle', 'su', 'sul', 'sullo', 'sui', 'sugli', 'sull', 'sugl', 'sulla', 'sulle', 'per', 'tra', 'contro', 'io', 'tu', 'lui', 'lei', 'noi', 'voi', 'loro', 'mio', 'mia', 'miei', 'mie', 'tuo', 'tua', 'tuoi', 'tue', 'suo', 'sua', 'suoi', 'sue', 'nostro', 'nostra', 'nostri', 'nostre', 'vostro', 'vostra', 'vostri', 'vostre', 'mi', 'ti', 'ci', 'vi', 'lo', 'la', 'li', 'le', 'gli', 'ne', 'il', 'un', 'uno', 'una', 'ma', 'ed', 'se', 'perché', 'anche', 'come', 'dov', 'dove', 'che', 'chi', 'cui', 'non', 'più', 'quale', 'quanto', 'quanti', 'quanta', 'quante', 'quello', 'quelli', 'quella', 'quelle', 'questo', 'q

In [4]:
# Percorso della cartella
folder_path = "./input"
def stampa_elenco_file(folder_path):
    # Lista dei file Excel nella cartella
    excel_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx") or f.endswith(".xls")]

    # Controlla se ci sono file Excel nella cartella
    if excel_files:
        print("Ecco l'elenco dei file Excel presenti nella cartella:")
        for index, file in enumerate(excel_files, start=1):
            print(f"{index}. {file}")
        return excel_files
    else:
        print("Nessun file Excel trovato nella cartella specificata.")
        return []

# Stampa l'elenco dei file e ottieni la lista dei file excel
excel_files = stampa_elenco_file(folder_path)

Ecco l'elenco dei file Excel presenti nella cartella:
1. keywords_20231018_1736.xlsx
2. short.xlsx
3. medium_30k.xlsx


In [5]:
def user_input_file_selection(excel_files, folder_path):
    # Richiedi input utente per selezionare un file
    while True:
        try:
            file_number = int(input("Inserisci il numero corrispondente al file da caricare: "))
            if 1 <= file_number <= len(excel_files):
                filename = os.path.join(folder_path, excel_files[file_number - 1])
                xl = pd.ExcelFile(filename)
                return xl, filename
            else:
                print("Il numero inserito non corrisponde a nessun file. Riprova.")
        except ValueError:
            print("Si prega di inserire un numero valido.")

# Se ci sono file Excel nella cartella, richiedi all'utente di selezionare un file
if excel_files:
    xl, filename = user_input_file_selection(excel_files, folder_path)

    # Ottieni la lista dei nomi dei fogli del file Excel selezionato
    nomi_fogli = xl.sheet_names
#    print("Ecco i nomi dei fogli presenti nel file Excel selezionato:")
#    for nome_foglio in nomi_fogli:
#        print(nome_foglio)

In [6]:
# stampa i nomi dei fogli
print("I fogli disponibili sono:")
for i, nome in enumerate(nomi_fogli):
    print(f"{i + 1}. {nome}")
   

I fogli disponibili sono:
1. pvt
2. Cluster
3. sistrix_liste


In [7]:
# chiedi all'utente di selezionare un foglio da leggere
while True:
    try:
        indice_foglio = int(input("Inserisci il numero del foglio che desideri leggere: ")) - 1
        if 0 <= indice_foglio < len(nomi_fogli):
            nome_foglio = nomi_fogli[indice_foglio]
            break
        else:
            print("Il numero inserito non è valido. Riprova.")
    except ValueError:
        print("Inserisci un numero valido. Riprova.")
    except Exception as e:
        print(f"Si è verificato un errore: {e}. Riprova.")

# leggi il foglio selezionato dall'utente
df = pd.read_excel(filename, sheet_name=nome_foglio)
df.head(5)
print('foglio letto')


foglio letto


In [8]:
#df.head(5)

In [9]:
# Crea una copia del dataframe originale
df1 = df.copy()

# Mostra all'utente le colonne disponibili nel dataframe
print("Here are the columns available in the dataframe:")
for i, col in enumerate(df1.columns):
    print(f"{i}: {col}")

Here are the columns available in the dataframe:
0: Keywords


In [10]:
# Chiedi all'utente di selezionare la colonna da mantenere
colonna_index = input("Enter the number of the column that contains the list of keywords: ")

# Continua a chiedere all'utente di inserire una colonna finché non viene fornita un'input valida
while not colonna_index.isdigit() or int(colonna_index) >= len(df1.columns):
    colonna_index = input("Invalid input. Please enter a valid column number: ")

colonna_da_mantenere = df1.columns[int(colonna_index)]
print(f'{len(df1)} parole chiave importate')

33245 parole chiave importate


In [11]:
# Cancella tutte le colonne tranne quella selezionata dall'utente
df1 = df1[[colonna_da_mantenere]]

# Rinomina la colonna
df1.rename(columns={colonna_da_mantenere: "Keywords"}, inplace=True)
print('foglio rinominato')

# Rimuovi le righe con valori nulli o vuoti
df1 = df1[df1['Keywords'].notnull() & (df1['Keywords'].str.strip() != '')]
print('righe vuote, Null e NaN rimosse')

# Togli duplicazioni
df1.drop_duplicates(subset='Keywords', keep='first', inplace=True)
print('duplicazioni rimosse')

# Stampa il dataframe risultante
df1.head(3)


foglio rinominato
righe vuote, Null e NaN rimosse
duplicazioni rimosse


,Keywords
0,07zr gomme
1,07zr pneumatici
2,07zr pneumatici recensioni


In [12]:
# pulizia parole chiave


# Funzione per pulire le parole chiave
def clean_keyword(keyword):
    # Convertire in minuscolo
    keyword = keyword.lower()
    # Sostituire la punteggiatura con uno spazio
    keyword = re.sub(r'[^\w\s]', ' ', keyword)
    # Rimuovere gli accenti
    keyword = unidecode(keyword)
    return keyword

# Creare la colonna 'Cleaned'
df1['Cleaned'] = df1['Keywords'].apply(clean_keyword)

df1.head(3)

,Keywords,Cleaned
0,07zr gomme,07zr gomme
1,07zr pneumatici,07zr pneumatici
2,07zr pneumatici recensioni,07zr pneumatici recensioni


In [13]:
# conta caratteri
df1c=df1.copy()
# Questa riga sostituisce tutti i valori NaN nella colonna Cleaned con una stringa vuota 
df1c['Cleaned'] = df1c['Cleaned'].fillna('')
# Questa riga converte tutti i valori nella colonna Cleaned in stringhe.
df1c['Cleaned'] = df1c['Cleaned'].apply(str)
# conteggio
df1c['Char'] = df1c['Cleaned'].apply(len)

print('colonna conta caratteri creata')
df2=df1c.copy()


colonna conta caratteri creata


In [14]:
# assegna alla colonna 'Without stopwords' del dataframe df2 il contenuto della PRIMA (Keywords) colonna del dataframe df, convertendola in stringa
#df2['Without stopwords'] = df2.iloc[:, 0].astype(str)

# assegna alla colonna 'Without stopwords' del dataframe df2 il contenuto della SECONDA (cleaned) colonna del dataframe df, convertendola in stringa
df2['Without stopwords'] = df2.iloc[:, 0].astype(str)

# rimuovi le stopwords dalla nuova colonna
df2['Without stopwords'] = df2['Without stopwords'].apply(
    lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords]))

df2
print('colonna senza stopword creata')

colonna senza stopword creata


In [15]:
# rimuovi termini indicati
# Chiedi all'utente di inserire le parole chiave da escludere, separate da virgola
exclude_words = input("If you want, enter keywords to be excluded from the classification, separated by comma: ").lower().split(",")
exclude_words = [word.strip() for word in exclude_words]
# Crea una copia della colonna "KW without stopwords"
df2['Without removed'] = df2['Without stopwords']
# Rimuovi le parole chiave escluse dalla nuova colonna
df2['Without removed'] = df2['Without removed'].apply(
    lambda x: ' '.join([word for word in x.split() if word.lower() not in exclude_words]))
# Visualizza il DataFrame con la nuova colonna
df2
print('colonna senza rimosse creata')

colonna senza rimosse creata


In [16]:
# stemming
df3=df2.copy()
# crea una copia della colonna "KW without stopwords"
df3['Stemming'] = df3['Without removed']
# esegui la stemming su ogni parola nella nuova colonna
df3['Stemming'] = df3['Stemming'].apply(
    lambda x: ' '.join([stemmer.stem(word) for word in x.split() if word.lower() not in stopwords]))
df3.head(3)

,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension


In [17]:
#Ordered by frequency
df4=df3.copy()
# crea una lista con tutti gli stemmi presenti nella colonna "Long Tail senza stopwords stemmata"
stem_list = []
for row in df4['Stemming']:
    stem_list.extend(row.split())
# conta le occorrenze di ogni stemma nella lista
stem_counts = Counter(stem_list)
# crea una lista di tuple (stemma, frequenza) ordinata in base alla frequenza decrescente
sorted_stems = sorted(stem_counts.items(), key=lambda x: x[1], reverse=True)
# crea un dizionario per mappare gli stemmi alle loro posizioni nella lista ordinata
stem_position_dict = {}
for i, (stem, _) in enumerate(sorted_stems):
    stem_position_dict[stem] = i
# crea la nuova colonna ordinando gli stemmi per frequenza assoluta
df4['Ordered by frequency'] = df4['Stemming'].apply(
    lambda x: ' '.join(sorted(x.split(), key=lambda s: stem_position_dict[s])))

print('colonna stemmi ordinati creata')
df4.head(3)

colonna stemmi ordinati creata


,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr


In [18]:
#Strongest word stem
df5=df4.copy()
# stemma più frequente try ok eng ok it
# creo una nuova colonna chiamata "Stemma più frequente"
df5['Strongest word stem'] = df5['Ordered by frequency'].str.split().str[0]

df5.head(3)

,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency,Strongest word stem
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr,gomm
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr,pneumat
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr,pneumat


In [19]:
# Calcola la frequenza di ogni parola in Strongest word stem
word_freq = df5['Strongest word stem'].value_counts(normalize=True)

# Calcola il valore percentuale cumulativo di ogni parola in Strongest word stem
word_cum_pct = word_freq.cumsum()

# Classifica la forza della parola in Strongest word stem in base al valore percentuale cumulativo
print('classificazione hi med low in corso')
def classify_word_strength(word):
    try:
        if word_cum_pct[word] <= 0.6:  # frequenza alta: "hi"
            return "hi"
        elif word_cum_pct[word] <= 0.8:  # frequenza media: "med"
            return "med"
        else:  # frequenza bassa: "low"
            return "low"
    except KeyError:
        return "unknown"

df5['Word Strength'] = df5['Strongest word stem'].apply(classify_word_strength)

print('classificazione hi med low termianta')
df5.head(3)

classificazione hi med low in corso
classificazione hi med low termianta


,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency,Strongest word stem,Word Strength
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr,gomm,low
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr,pneumat,med
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr,pneumat,med


In [20]:
# stemmi
df6=df5.copy()
# Crea la nuova colonna "2 Stemmi più frequente" con i due stemmi più frequenti
df6['Strongest 2 words stems'] = df6['Ordered by frequency'].str.split().str[:2].str.join(' ')
df6.loc[df6['Strongest 2 words stems'].str.count(' ') < 1, 'Strongest 2 words stems'] = ''
print('classificazione 2x stemmi')
df7=df6.copy()
df7['Strongest 3 words stems'] = df7['Ordered by frequency'].str.split().str[:3].str.join(' ')
df7.loc[df7['Strongest 3 words stems'].str.count(' ') < 2, 'Strongest 3 words stems'] = ''
print('classificazione 3x stemmi')
df8 = df7.copy()
df8['Strongest 4 words stems'] = df7['Ordered by frequency'].str.split().str[:4].str.join(' ')
df8.loc[df8['Strongest 4 words stems'].str.count(' ') < 3, 'Strongest 4 words stems'] = ''
print('classificazione 4x stemmi')
df8.head(3)

classificazione 2x stemmi
classificazione 3x stemmi
classificazione 4x stemmi


,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency,Strongest word stem,Word Strength,Strongest 2 words stems,Strongest 3 words stems,Strongest 4 words stems
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr,gomm,low,gomm 07zr,,
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr,pneumat,med,pneumat 07zr,,
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr,pneumat,med,pneumat recension,pneumat recension 07zr,


In [21]:
# SPACY VERBI NOMI AGGETTIVI
dfspacy = df8.copy()

print('classificazione Spacy per VERBI NOMI AGGETTIVI avviata')
# Crea le liste per conservare le categorie delle parole chiave, VERB e NOUN e ADJ e NOUN
spacy_cat = []
spacy_lemma = []
verb_noun_cat = []
adj_noun_cat = []
vadj_noun_cat = []

# Analizza ogni riga del DataFrame con Spacy
for row in tqdm(dfspacy['Keywords']):
#for row in df2vec['Keywords']:
    doc = nlp(row)
    category = None
    lemma = None
    verb_noun = []
    adj_noun = []
    vadj_noun = []
  
    # Estrae le categorie e lemmi di NOUN dalla riga
    for token in doc:
        if token.pos_ in ['NOUN']:
            #Se il token è un sostantivo, il suo testo viene assegnato alla variabile category.
            category = token.text
            # Il "lemma" (forma base) del token viene assegnato alla variabile lemma.
            lemma = token.lemma_
        if token.pos_ in ['VERB', 'NOUN']:
            verb_noun.append(token.text)
        if token.pos_ in ['ADJ', 'NOUN']:
            adj_noun.append(token.text)
        if token.pos_ in ['ADJ', 'VERB']:
            vadj_noun.append(token.text)
    
    # Assegna una categoria predefinita se non è stata identificata nessuna categoria
    if category is None:
        category = 'other'
    if lemma is None:
        lemma = 'other'
    
    # Aggiungi le categorie e lemmi alla lista
    spacy_cat.append(category)
    spacy_lemma.append(lemma)
    
    # Aggiungi le categorie VERB e NOUN e ADJ e NOUN alle liste
    verb_noun_cat.append(', '.join(verb_noun))
    adj_noun_cat.append(', '.join(adj_noun))
    vadj_noun_cat.append(', '.join(vadj_noun))

# Aggiungi le liste delle categorie come nuove colonne del DataFrame
dfspacy['spacy_cat'] = spacy_cat
dfspacy['spacy_lemma'] = spacy_lemma
dfspacy['verb_noun_cat'] = verb_noun_cat
dfspacy['adj_noun_cat'] = adj_noun_cat
dfspacy['verb_adj_cat'] = vadj_noun_cat

print('classificazione Spacy terminata')

classificazione Spacy per VERBI NOMI AGGETTIVI avviata


  0%|          | 0/33189 [00:00<?, ?it/s]

100%|██████████| 33189/33189 [04:11<00:00, 131.95it/s]

classificazione Spacy terminata


In [22]:
# Visualizza il dataframe con le categorie assegnate
print("Numero di righe:", len(df))

dfspacy.head(3)

Numero di righe: 33245


,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency,Strongest word stem,Word Strength,Strongest 2 words stems,Strongest 3 words stems,Strongest 4 words stems,spacy_cat,spacy_lemma,verb_noun_cat,adj_noun_cat,verb_adj_cat
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr,gomm,low,gomm 07zr,,,gomme,gomma,gomme,"07zr, gomme",07zr
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr,pneumat,med,pneumat 07zr,,,pneumatici,pneumatico,"07zr, pneumatici",pneumatici,07zr
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr,pneumat,med,pneumat recension,pneumat recension 07zr,,recensioni,recensione,"07zr, pneumatici, recensioni","pneumatici, recensioni",07zr


In [23]:
# detect language
print('classificazione LANG iniziata')
dflang = dfspacy.copy()

# Impostazione del batch size
batch_size = 1000

print(f"Inizio della classificazione LANG in blocchi da {batch_size} parole")

# Inizializzazione della lista vuota per conservare i risultati
langs = []

# Calcolo del numero di batch
num_batches = len(dflang) // batch_size + (len(dflang) % batch_size != 0)

# Loop attraverso i batch
for i in tqdm(range(num_batches), desc="Linguaggio"):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    
    # Estrai il batch corrente dal DataFrame
    batch = dflang['Cleaned'][start_idx:end_idx]
    
    # Loop attraverso le parole chiave nel batch
    for x in batch:
        if len(x) > 0 and any(c.isalpha() for c in x):
            try:
                lang = detect(x)
            except:
                lang = ''
        else:
            lang = ''
        langs.append(lang)

# Aggiungi i risultati al DataFrame
dflang['Lang'] = langs

print('Classificazione LANG completata')
dflang.head(3)

classificazione LANG iniziata
Inizio della classificazione LANG in blocchi da 1000 parole


Linguaggio:   0%|          | 0/34 [00:00<?, ?it/s]

Linguaggio: 100%|██████████| 34/34 [09:00<00:00, 15.89s/it]

Classificazione LANG completata


,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency,Strongest word stem,Word Strength,Strongest 2 words stems,Strongest 3 words stems,Strongest 4 words stems,spacy_cat,spacy_lemma,verb_noun_cat,adj_noun_cat,verb_adj_cat,Lang
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr,gomm,low,gomm 07zr,,,gomme,gomma,gomme,"07zr, gomme",07zr,hr
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr,pneumat,med,pneumat 07zr,,,pneumatici,pneumatico,"07zr, pneumatici",pneumatici,07zr,hr
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr,pneumat,med,pneumat recension,pneumat recension 07zr,,recensioni,recensione,"07zr, pneumatici, recensioni","pneumatici, recensioni",07zr,it


In [24]:
# spaCy similarity - eligible

dfelig=dflang.copy()
# Creare un modello di similarità del testo utilizzando tutte le parole chiave
keywords_text = ' '.join(dfelig['Keywords'])
nlp.max_length = 6000000  # or any large number suitable to your hardware, circa 1 ML di parole
keywords_nlp = nlp(keywords_text)

# Funzione per valutare l'idoneità di una parola chiave
def evaluate_idoneity(keyword):
    keyword_nlp = nlp(keyword)
    similarity = keyword_nlp.similarity(keywords_nlp)
    return 'Eligible' if similarity > 0.2 else 'Not eligible'  # Soglia di similarità, regolabile

# Applicare la funzione al DataFrame
#dfL_w2v['Eligible'] = dfL_w2v['Keywords'].apply(evaluate_idoneity)
#tqdm version
tqdm.pandas()
dfelig['NLP Eligible'] = dfelig['Keywords'].progress_apply(evaluate_idoneity)

# Visualizzare il DataFrame con la nuova colonna "Idoneity"
print("Numero di righe:", len(df))

dfelig.head(3)

  0%|          | 16/33189 [00:00<17:28, 31.64it/s] C:\Users\Giovanni Sacheli\AppData\Local\Temp\ipykernel_21544\3471795252.py:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = keyword_nlp.similarity(keywords_nlp)
100%|██████████| 33189/33189 [04:37<00:00, 119.46it/s]

Numero di righe: 33245


,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency,Strongest word stem,Word Strength,Strongest 2 words stems,Strongest 3 words stems,Strongest 4 words stems,spacy_cat,spacy_lemma,verb_noun_cat,adj_noun_cat,verb_adj_cat,Lang,NLP Eligible
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr,gomm,low,gomm 07zr,,,gomme,gomma,gomme,"07zr, gomme",07zr,hr,Eligible
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr,pneumat,med,pneumat 07zr,,,pneumatici,pneumatico,"07zr, pneumatici",pneumatici,07zr,hr,Eligible
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr,pneumat,med,pneumat recension,pneumat recension 07zr,,recensioni,recensione,"07zr, pneumatici, recensioni","pneumatici, recensioni",07zr,it,Eligible


In [25]:
# Bert Eligible
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Caricare un modello Sentence-BERT specifico per l'italiano
print('loading model')
#model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

print('data loading')
# Supponiamo che dfelig sia il tuo DataFrame e stai usando la colonna 'Keywords'
keywords = dfelig['Without stopwords'].tolist()

print('embedding')
# Generare embeddings per tutte le parole chiave
embeddings = model.encode(keywords, convert_to_tensor=True)

print('similarity matrix')
# Calcolare la matrice di similarità
similarity_matrix = cosine_similarity(embeddings)

print('anomalies detection')
# Identificare le parole chiave anomale (ad esempio, quelle con una bassa media di similarità)
mean_similarity = np.mean(similarity_matrix, axis=1)
threshold = np.quantile(mean_similarity, 0.1)  # il 10% più basso della similarità media potrebbe essere considerato anomalo
dfelig['BERT Eligible'] = ['Eligible' if sim > threshold else 'Not eligible' for sim in mean_similarity]

# Visualizzare il DataFrame con la nuova colonna "Eligible"
print("Numero di righe:", len(dfelig))
dfelig.head(3)


e:\python\ambiente\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading model
data loading
embedding
similarity matrix
anomalies detection
Numero di righe: 33189


,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency,Strongest word stem,Word Strength,Strongest 2 words stems,Strongest 3 words stems,Strongest 4 words stems,spacy_cat,spacy_lemma,verb_noun_cat,adj_noun_cat,verb_adj_cat,Lang,NLP Eligible,BERT Eligible
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr,gomm,low,gomm 07zr,,,gomme,gomma,gomme,"07zr, gomme",07zr,hr,Eligible,Not eligible
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr,pneumat,med,pneumat 07zr,,,pneumatici,pneumatico,"07zr, pneumatici",pneumatici,07zr,hr,Eligible,Eligible
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr,pneumat,med,pneumat recension,pneumat recension 07zr,,recensioni,recensione,"07zr, pneumatici, recensioni","pneumatici, recensioni",07zr,it,Eligible,Eligible


In [26]:
# dbscan cluster

from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np


dfdbscan=dfelig.copy()
# Create a mock DataFrame named dfelig

# Prepare data for Word2Vec model training
dfdbscan['Split_Words'] = dfdbscan['Cleaned'].apply(lambda x: x.split())

# Train the Word2Vec model
model = Word2Vec(sentences=dfdbscan['Split_Words'], vector_size=100, window=5, min_count=1, workers=4)
model.train(dfdbscan['Split_Words'], total_examples=len(dfdbscan['Split_Words']), epochs=10)

# Transform keywords into vectors
def phrase_to_vec(phrase):
    words = phrase.split()
    vectors = [model.wv[word] for word in words if word in model.wv.index_to_key]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.nan

dfdbscan['Vector'] = dfdbscan['Cleaned'].apply(phrase_to_vec)

# Remove rows where the phrase could not be converted to a vector
dfdbscan.dropna(subset=['Vector'], inplace=True)

# Transform the vectors into a NumPy array for DBSCAN
vectors = np.stack(dfdbscan['Vector'].to_numpy())

# Standardize features to have zero mean and unit variance
scaler = StandardScaler()
vectors = scaler.fit_transform(vectors)

# Apply DBSCAN clustering
clustering = DBSCAN(eps=0.5, min_samples=2).fit(vectors)

# Add the labels to the DataFrame
dfdbscan['Word2Vec_DBSCAN Cluster'] = clustering.labels_
print("Numero di righe:", len(df))

dfdbscan.head(3)


Numero di righe: 33245


,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency,Strongest word stem,Word Strength,Strongest 2 words stems,...,spacy_lemma,verb_noun_cat,adj_noun_cat,verb_adj_cat,Lang,NLP Eligible,BERT Eligible,Split_Words,Vector,Word2Vec_DBSCAN Cluster
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr,gomm,low,gomm 07zr,...,gomma,gomme,"07zr, gomme",07zr,hr,Eligible,Not eligible,"[07zr, gomme]","[-0.6021453, 0.3987788, 0.04441467, 0.1367907,...",0
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr,pneumat,med,pneumat 07zr,...,pneumatico,"07zr, pneumatici",pneumatici,07zr,hr,Eligible,Eligible,"[07zr, pneumatici]","[-0.09687947, -0.18365835, -0.079624794, 0.353...",1
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr,pneumat,med,pneumat recension,...,recensione,"07zr, pneumatici, recensioni","pneumatici, recensioni",07zr,it,Eligible,Eligible,"[07zr, pneumatici, recensioni]","[-0.14097951, 0.10361303, -0.2190255, 0.392285...",2


In [27]:
# BERT HDBSCAN

dfbert = dfdbscan.copy()

from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import hdbscan
from tqdm.auto import tqdm


# Caricare il modello BERT pre-addestrato per l'italiano
model_name = "dbmdz/bert-base-italian-xxl-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Supponiamo che df sia il tuo dataframe e stai usando la colonna 'Keywords'
keywords = dfbert['Without stopwords'].tolist()	

# Tokenizzazione e creazione degli embeddings
tokens = tokenizer(keywords, padding=True, truncation=True, return_tensors="pt")

input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

# Definire la dimensione del batch
batch_size = 1000  # Aggiusta questo numero in base alla capacità della tua CPU

# con tqdm
embeddings = []
for i in tqdm(range(0, input_ids.size(0), batch_size), desc="Processing"):
    batch_input_ids = input_ids[i:i+batch_size]
    batch_attention_mask = attention_mask[i:i+batch_size]
    
    with torch.no_grad():
        batch_outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        batch_embeddings = batch_outputs.last_hidden_state.mean(dim=1)
        embeddings.append(batch_embeddings)

# Concatenare gli embeddings per ottenere un unico tensore
embeddings = torch.cat(embeddings, dim=0)



Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Processing: 100%|██████████| 34/34 [13:36<00:00, 24.00s/it]


Clustering
df updating


In [28]:
# Applicare HDBSCAN per il clustering
print('Clustering')
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, gen_min_span_tree=True, core_dist_n_jobs=-1)
print('df updating')
dfbert['Bert_Hdbscan Cluster'] = clusterer.fit_predict(embeddings.numpy())
dfbert

,Keywords,Cleaned,Char,Without stopwords,Without removed,Stemming,Ordered by frequency,Strongest word stem,Word Strength,Strongest 2 words stems,...,verb_noun_cat,adj_noun_cat,verb_adj_cat,Lang,NLP Eligible,BERT Eligible,Split_Words,Vector,Word2Vec_DBSCAN Cluster,Bert_Hdbscan Cluster
0,07zr gomme,07zr gomme,10,07zr gomme,07zr gomme,07zr gomm,gomm 07zr,gomm,low,gomm 07zr,...,gomme,"07zr, gomme",07zr,hr,Eligible,Not eligible,"[07zr, gomme]","[-0.6021453, 0.3987788, 0.04441467, 0.1367907,...",0,-1
1,07zr pneumatici,07zr pneumatici,15,07zr pneumatici,07zr pneumatici,07zr pneumat,pneumat 07zr,pneumat,med,pneumat 07zr,...,"07zr, pneumatici",pneumatici,07zr,hr,Eligible,Eligible,"[07zr, pneumatici]","[-0.09687947, -0.18365835, -0.079624794, 0.353...",1,-1
2,07zr pneumatici recensioni,07zr pneumatici recensioni,26,07zr pneumatici recensioni,07zr pneumatici recensioni,07zr pneumat recension,pneumat recension 07zr,pneumat,med,pneumat recension,...,"07zr, pneumatici, recensioni","pneumatici, recensioni",07zr,it,Eligible,Eligible,"[07zr, pneumatici, recensioni]","[-0.14097951, 0.10361303, -0.2190255, 0.392285...",2,-1
3,10 0 75 pneumatici alliance trattore,10 0 75 pneumatici alliance trattore,36,10 0 75 pneumatici alliance trattore,10 0 75 pneumatici alliance trattore,10 0 75 pneumat allianc trattor,pneumat trattor 75 10 allianc 0,pneumat,med,pneumat trattor,...,"pneumatici, trattore","pneumatici, trattore",,it,Eligible,Eligible,"[10, 0, 75, pneumatici, alliance, trattore]","[-0.2504684, 0.40147385, -0.28380698, 0.454443...",3,-1
4,10.0 75 pneumatici alliance trattore,10 0 75 pneumatici alliance trattore,36,10.0 75 pneumatici alliance trattore,10.0 75 pneumatici alliance trattore,10.0 75 pneumat allianc trattor,pneumat trattor 75 allianc 10.0,pneumat,med,pneumat trattor,...,"10.0, pneumatici, trattore","pneumatici, trattore",10.0,it,Eligible,Eligible,"[10, 0, 75, pneumatici, alliance, trattore]","[-0.2504684, 0.40147385, -0.28380698, 0.454443...",3,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33240,pneumatici 4 stagioni malatesta,pneumatici 4 stagioni malatesta,31,pneumatici 4 stagioni malatesta,pneumatici 4 stagioni malatesta,pneumat 4 stagion malatest,pneumat stagion 4 malatest,pneumat,med,pneumat stagion,...,"pneumatici, stagioni","pneumatici, stagioni, malatesta",malatesta,it,Eligible,Eligible,"[pneumatici, 4, stagioni, malatesta]","[-0.37670502, 0.3059896, -0.58144176, 0.470484...",814,-1
33241,pneumatici 4 stagioni marche,pneumatici 4 stagioni marche,28,pneumatici 4 stagioni marche,pneumatici 4 stagioni marche,pneumat 4 stagion marc,pneumat stagion 4 marc,pneumat,med,pneumat stagion,...,"pneumatici, stagioni","pneumatici, stagioni, marche",marche,it,Eligible,Eligible,"[pneumatici, 4, stagioni, marche]","[-0.35126996, 0.363717, -0.5317086, 0.5477126,...",2350,1496
33242,pneumatici 4 stagioni marshal,pneumatici 4 stagioni marshal,29,pneumatici 4 stagioni marshal,pneumatici 4 stagioni marshal,pneumat 4 stagion marshal,pneumat stagion 4 marshal,pneumat,med,pneumat stagion,...,"pneumatici, stagioni","pneumatici, stagioni",,it,Eligible,Eligible,"[pneumatici, 4, stagioni, marshal]","[-0.3741749, 0.2754958, -0.57852155, 0.4731719...",814,-1
33243,pneumatici 4 stagioni matador,pneumatici 4 stagioni matador,29,pneumatici 4 stagioni matador,pneumatici 4 stagioni matador,pneumat 4 stagion matador,pneumat stagion 4 matador,pneumat,med,pneumat stagion,...,"pneumatici, stagioni","pneumatici, stagioni, matador",matador,it,Eligible,Eligible,"[pneumatici, 4, stagioni, matador]","[-0.38435698, 0.29144254, -0.5759673, 0.487256...",814,-1


In [29]:
# export to excel
dataframes = {
    'dfbert': 'Cluster'
}
print('exporting...')

# ottieni la data e l'ora corrente
now = datetime.now()
# formatta il timestamp nel formato desiderato
timestamp = now.strftime("%Y%m%d_%H%M")
filename = f'{timestamp}_CLUSTER_{sorted_stems[0][0]}.xlsx'

cartella_output = "./output"
with pd.ExcelWriter(os.path.join(cartella_output, filename)) as writer:
#with pd.ExcelWriter(os.path.join(output_dir, f'{sorted_stems[0][0]}.xlsx')) as writer:
    for df_name, sheet_name in tqdm(dataframes.items(), total=len(dataframes), desc="Saving Dataframes"):
        if df_name in locals():
            locals()[df_name].to_excel(writer, sheet_name=sheet_name, index=True)
print(f'\n{filename}.xlsx exported and ready for download in {cartella_output}')

exporting...


Saving Dataframes: 100%|██████████| 1/1 [00:23<00:00, 23.62s/it]



20231102_2129_CLUSTER_pneumat.xlsx.xlsx exported and ready for download in ./output
